# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [163]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [164]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [165]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @gubenites99

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

FileNotFoundError: [Errno 2] No such file or directory: 'auth.pass'

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [ ]:
#Produto escolhido:
produto = 'narcos'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [ ]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [ ]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [276]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import json
import tweepy
from random import shuffle
import sys
import mpmath


exl=pd.ExcelFile("narcos.xlsx")
exl.sheet_names
[u'Treinamento', u'Teste']
twits = exl.parse("Treinamento")
twits2 = exl.parse("Teste")

twits2.head()

,Teste,Relevancia
0,rt @huffpostbrasil: produtor de 'narcos' é ass...,Não
1,rt @filipe_afc: o produtor da série narcos foi...,Não
2,rt @smalivee: melhor abertura de séries é a de...,Não
3,a minha vida ta um loop da musica da abertura ...,Sim
4,rt @comculturaearte: scout de ‘narcos’ morre n...,Não


In [277]:
twits['Treinamento']=twits['Treinamento'].str.replace(':','')
twits['Treinamento']=twits['Treinamento'].str.replace(',','')
twits['Treinamento']=twits['Treinamento'].str.replace("'",'')
twits['Treinamento']=twits['Treinamento'].str.replace('-','')
twits['Treinamento']=twits['Treinamento'].str.replace('"','')
twits['Treinamento']=twits['Treinamento'].str.replace(')','')
twits['Treinamento']=twits['Treinamento'].str.replace('(','')
twits['Treinamento']=twits['Treinamento'].str.replace(' é ',' ')
twits['Treinamento']=twits['Treinamento'].str.replace(' a ',' ')
twits['Treinamento']=twits['Treinamento'].str.replace(' o ',' ')
twits['Treinamento']=twits['Treinamento'].str.replace(' e ',' ')
twits['Treinamento']=twits['Treinamento'].str.replace('rt','')
twits['Treinamento']=twits['Treinamento'].str.replace(' os ',' ')
twits['Treinamento']=twits['Treinamento'].str.replace(' as ',' ')
twits['Treinamento']=twits['Treinamento'].str.replace(' ou ',' ')
twits['Treinamento']=twits['Treinamento'].str.replace('  ',' ')
twits['Treinamento']=twits['Treinamento'].str.replace(' que ',' ')
twits['Treinamento']=twits['Treinamento'].str.replace(' nas ',' ')
twits['Treinamento']=twits['Treinamento'].str.replace(' na ',' ')
twits['Treinamento']=twits['Treinamento'].str.replace(' no ',' ')
twits['Treinamento']=twits['Treinamento'].str.replace(' nos ',' ')
twits['Treinamento']=twits['Treinamento'].str.replace('?','')
twits['Treinamento']=twits['Treinamento'].str.replace('/','')
twits['Treinamento']=twits['Treinamento'].str.replace('@','')
twits['Treinamento']=twits['Treinamento'].str.replace('!','')
twits['Treinamento']=twits['Treinamento'].str.replace('|','')
twits['Treinamento'] = twits['Treinamento'].str.lower()

twits.Treinamento.head()

0     roselyrooms bimbi pode ser espanadoragressor....
1     sicnoticias assistente de produção da série n...
2                    v fazer esse continuar ver narcos
3                            acabei got comecei narcos
4    vou aí ver mais um episódio de narcos antes de...
Name: Treinamento, dtype: object

In [278]:
twits2['Teste']=twits2['Teste'].str.replace(':','')
twits2['Teste']=twits2['Teste'].str.replace(',','')
twits2['Teste']=twits2['Teste'].str.replace("'",'')
twits2['Teste']=twits2['Teste'].str.replace('-','')
twits2['Teste']=twits2['Teste'].str.replace('"','')
twits2['Teste']=twits2['Teste'].str.replace(')','')
twits2['Teste']=twits2['Teste'].str.replace('(','')
twits2['Teste']=twits2['Teste'].str.replace(' é ',' ')
twits2['Teste']=twits2['Teste'].str.replace(' a ',' ')
twits2['Teste']=twits2['Teste'].str.replace(' o ',' ')
twits2['Teste']=twits2['Teste'].str.replace(' e ',' ')
twits2['Teste']=twits2['Teste'].str.replace('rt','')
twits2['Teste']=twits2['Teste'].str.replace(' os ',' ')
twits2['Teste']=twits2['Teste'].str.replace(' as ',' ')
twits2['Teste']=twits2['Teste'].str.replace(' ou ',' ')
twits2['Teste']=twits2['Teste'].str.replace('  ',' ')
twits2['Teste']=twits2['Teste'].str.replace(' que ',' ')
twits2['Teste']=twits2['Teste'].str.replace(' nas ',' ')
twits2['Teste']=twits2['Teste'].str.replace(' na ',' ')
twits2['Teste']=twits2['Teste'].str.replace(' no ',' ')
twits2['Teste']=twits2['Teste'].str.replace(' nos ',' ')
twits2['Teste']=twits2['Teste'].str.replace('?','')
twits2['Teste']=twits2['Teste'].str.replace('/','')
twits2['Teste']=twits2['Teste'].str.replace('@','')
twits2['Teste']=twits2['Teste'].str.replace('!','')
twits2['Teste']=twits2['Teste'].str.replace('|','')
twits2['Teste'] = twits2['Teste'].str.lower()

twits2.Teste.head()

0     huffpostbrasil produtor de narcos assassinado...
1     filipe_afc produtor da série narcos foi encon...
2     smalivee melhor abeura de séries de narcos ai...
3    a minha vida ta um loop da musica da abeura de...
4     comculturaeae scout de ‘narcos’ morre trabalh...
Name: Teste, dtype: object

In [279]:
relevante = []
naorelevante = []
naorelevante_semrepetiçao = []
relevante_semrepetiçao = []
all = []
all_semrepetiçao = []


for i in range(len(twits)): 
    all+=(list(twits["Treinamento"][i].split()))
all_semrepetiçao=(list(set(all)))

for i in range(len(twits)):
    if twits["Relevancia"][i]=="Sim":
        relevante+=(list(twits["Treinamento"][i].split()))

    if twits["Relevancia"][i]=="Não":
        naorelevante+=(list(twits["Treinamento"][i].split()))

naorelevante_semrepetiçao=list(set(naorelevante))
relevante_semrepetiçao=list(set(relevante))


numerogeralsemrepetições=len(all_semrepetiçao)
relevantecontagem=len(relevante_semrepetiçao)
naorelevantecontagem=len(naorelevante_semrepetiçao)


In [280]:
#Quantidade de palavras relevantes
Quantidadeprelevante={}
Quantidadepnaorelevante={}
for i in range(len(relevante)):
    if relevante[i] in Quantidadeprelevante:
        Quantidadeprelevante[relevante[i]]=Quantidadeprelevante[relevante[i]]+1
    else:
        Quantidadeprelevante[relevante[i]]=1
#Quantidade de palavras não relevantes       
for i in range(len(naorelevante)):
    if naorelevante[i] in Quantidadepnaorelevante:
        Quantidadepnaorelevante[naorelevante[i]]=Quantidadepnaorelevante[naorelevante[i]]+1
    else:
        Quantidadepnaorelevante[naorelevante[i]]=1


In [281]:
#Calculando probabilidade de cada palavra
probrelevantepalavra={}
probnaorelevantepalavra={}
for i in Quantidadeprelevante:
    probrelevantepalavra[i]=(Quantidadeprelevante[i]+1)/(numerogeralsemrepetições+relevantecontagem)
for i in Quantidadepnaorelevante:
    probnaorelevantepalavra[i]=(Quantidadepnaorelevante[i]+1)/(numerogeralsemrepetições+naorelevantecontagem)

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [282]:
frasesteste=[]
listarelevancia=[]
listavaloresfrasesRelevantes=[]
listavaloresfrasesNaorelevantes=[]
contadorrelvante=1
contadornaorelvante=1
for i in twits2['Teste']:
    a=i.split()
    frasesteste.append(a)
#Relevancia
for i in frasesteste:
    for h in i:
        if h in probrelevantepalavra:
            contadorrelvante=contadorrelvante*probrelevantepalavra[h]
        else:
            contadorrelvante=contadorrelvante*(1/(numerogeralsemrepetições+relevantecontagem))
    contadorrelvante=mpmath.mpf(contadorrelvante)
    listavaloresfrasesRelevantes.append(contadorrelvante)
    contadorrelvante=1
#Nao Relevante
for i in frasesteste:
    for h in i:
        if h in probnaorelevantepalavra:
            contadornaorelvante=contadornaorelvante*probnaorelevantepalavra[h]     
        else:
            contadorrelvante=contadorrelvante*(1/(numerogeralsemrepetições+naorelevantecontagem))
    contadornaorelvante=mpmath.mpf(contadornaorelvante)
    listavaloresfrasesNaorelevantes.append(contadornaorelvante)
    contadornaorelvante=1

listavaloresfrasesRelevantes

[mpf('3.5948777936433993e-40'),
 mpf('2.2666915855985006e-27'),
 mpf('4.9507519262177369e-23'),
 mpf('4.1958369433671219e-30'),
 mpf('1.9451016259486425e-20'),
 mpf('5.6742685571859924e-16'),
 mpf('4.0936450035908926e-22'),
 mpf('1.4589601435241062e-45'),
 mpf('3.4113708363257442e-23'),
 mpf('1.9451016259486425e-20'),
 mpf('6.0703659481584518e-34'),
 mpf('5.1423725501199748e-30'),
 mpf('4.8733869090367766e-26'),
 mpf('7.3204784161465535e-39'),
 mpf('6.9906988624531431e-11'),
 mpf('3.0357476592837061e-30'),
 mpf('7.7428137093857812e-39'),
 mpf('1.1001670947150526e-22'),
 mpf('7.8295827259475212e-8'),
 mpf('1.9840254699022875e-45'),
 mpf('4.8733869090367766e-26'),
 mpf('2.7195239447749875e-29'),
 mpf('1.728306631559326e-42'),
 mpf('6.9537383127663978e-19'),
 mpf('4.2792235770870143e-18'),
 mpf('1.232187146080859e-19'),
 mpf('3.2519817579420294e-36'),
 mpf('1.6443633154453411e-30'),
 mpf('4.4347820872234678e-23'),
 mpf('1.2280935010772681e-22'),
 mpf('1.2183467272591944e-26'),
 mpf('1.327

In [283]:
ListaRelevanciaTeste=[]
for i in range(len(listavaloresfrasesRelevantes)):
    if listavaloresfrasesRelevantes[i]>listavaloresfrasesNaorelevantes[i]:
        ListaRelevanciaTeste.append('Sim')
    if listavaloresfrasesRelevantes[i]<listavaloresfrasesNaorelevantes[i]:
        ListaRelevanciaTeste.append('Não')
    if listavaloresfrasesRelevantes[i]==listavaloresfrasesNaorelevantes[i]:
        ListaRelevanciaTeste.append('Neutro')
ListaRelevanciaTeste

['Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',
 'Não',


___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
